<a href="https://colab.research.google.com/github/m109103/bert/blob/main/MyBert_k10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=6b7ac18c65c812faf5ca1534dae379f9e7c6e5fbe94e2ff4ce1986ad6b96a8e2
  Stored in directory: /root/.cache/pip/wheels/a4/e8/45/842b3a39831261aef9154b907eacbc4ac99499a99ae829b06f
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12305 sha256=582f0f1fc4b4d7cbb28908e98f8955af46dc2d7b5fd2c1d05bda2d9102bbd1ab
  Stored in directory: /root/.cache/pip/wheels/46/68/26/692ed21edd832833c3b0a0e21615bcacd99ca458b3f9ed571f
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3960 sha256=4b009ec7bdd52127ab5ac8e94972fc07cf346c9f062ecf621b8e45cafc12261f
  Stored in directory: /root/.cache/pip/wheels/81/67/b5/d847588d075895281e1cf5590f819bd4cf076a554872268bd5
  Created wheel for keras-layer-normalization: filename=keras_layer_normalizatio

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 31 02:37:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
1.4297627200000136
GPU (s):
0.036829251999989765
GPU speedup over CPU: 38x


In [5]:
from google.colab import drive  
drive.mount('/content/gdrive')  
%cd /content/gdrive/"My Drive"/"Colab Notebooks"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


In [6]:
import sys
import random
import numpy as np

from tensorflow.keras.layers import *
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from keras_bert import (
    load_vocabulary,
    load_trained_model_from_checkpoint,
    Tokenizer,
    get_checkpoint_paths,
)
from keras_bert.datasets import get_pretrained, PretrainedList

In [7]:
# 指定訓練資料與測試資料的比例
BATCH_SIZE = 10 # batch size建議不要設得太大，不然很有可能out of memory
EPOCHS = 5 # epoch 5次其實就很夠了，當然你可以嘗試再大一點，只是訓練要更久

In [8]:
model_path = get_pretrained(PretrainedList.chinese_base)
paths = get_checkpoint_paths(model_path)
bert_model = load_trained_model_from_checkpoint(
    paths.config, paths.checkpoint, training=False, seq_len=None
)
bert_model.summary(line_length=120)

381902848/381892918 [==============================] - 5s 0us/step
Model: "model_1"
________________________________________________________________________________________________________________________
 Layer (type)                          Output Shape               Param #       Connected to                            
 Input-Token (InputLayer)              [(None, None)]             0             []                                      
                                                                                                                        
 Input-Segment (InputLayer)            [(None, None)]             0             []                                      
                                                                                                                        
 Embedding-Token (TokenEmbedding)      [(None, None, 768),        16226304      ['Input-Token[0][0]']                   
                                        (21128, 768)]                

In [9]:
token_dict = load_vocabulary(paths.vocab)
len(token_dict)
tokenizer = Tokenizer(token_dict)

載入打散後資料

In [10]:
import pickle
with open('./data/r_text.pickle', 'rb') as f:
    x_text = pickle.load(f)
with open('./data/r_tag.pickle', 'rb') as f:
    y_tag = pickle.load(f)

X = x_text[:]
Y = y_tag[:]
#len(x_text)=6317

In [11]:
#設置模型
def build_model():
    # 把 BERT 模型裡的每一層的trainable屬性都設成True，讓BERT模型裡的權重都可以被訓練
    for l in bert_model.layers:
        l.trainable = True

    # 模型的輸入，我們會需要兩個輸入，分別是語句內容以及其對應的segments
    x1_in = Input(shape=(None,))
    x2_in = Input(shape=(None,))
    x = bert_model([x1_in, x2_in])
    x = Lambda(lambda x: x[:, 0])(x) # 取出[CLS]對應位置的向量出來
    p = Dense(18, activation='softmax')(x) # 輸出情緒傾向數值

    model = Model([x1_in, x2_in], p)

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(1e-5),    #用較小的學習率
                  metrics=['accuracy'])

    #model.summary()
    return model

In [12]:
k=10
num_val_samples = len(X) // k
cvscores = []
results = []
for i in range(k):
    print('processing fold #',i)
    test_data = X[i*num_val_samples:(i+1)*num_val_samples]
    tag_test = Y[i*num_val_samples:(i+1)*num_val_samples]
    train_data = np.concatenate(
        [X[:i*num_val_samples],X[(i+1)*num_val_samples:]],axis=0)
    tag_train = np.concatenate(
        [Y[:i*num_val_samples],Y[(i+1)*num_val_samples:]],axis=0)

    partial_train_data = []
    partial_train_targets = []
    val_data = []
    val_targets = []

    indices = []
    for sentence in train_data:
        ids, segments = tokenizer.encode(sentence, max_len=300) # Tokenizer進行分詞
        indices.append(ids)
    indices = np.array(indices)
    partial_train_data = [indices, np.zeros_like(indices)]
    partial_train_targets = np.array(tag_train)

    indices = []
    for sentence in test_data:
        ids, segments = tokenizer.encode(sentence, max_len=300) # Tokenizer進行分詞
        indices.append(ids)
    indices = np.array(indices)
    val_data = [indices, np.zeros_like(indices)]
    val_targets = np.array(tag_test)
    
    model = build_model()   
    
    history = model.fit(
        partial_train_data,partial_train_targets,
        batch_size=BATCH_SIZE,epochs=EPOCHS,verbose=1)
# evaluate the model
    scores = model.evaluate(val_data,val_targets,batch_size=BATCH_SIZE, verbose=1) 
    result = model.predict(val_data)
    results.append(result)
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("End of Predict DataSet!")

processing fold # 0
Epoch 1/5
569/569 [==============================] - 279s 456ms/step - loss: 1.0321 - accuracy: 0.7019
Epoch 2/5
569/569 [==============================] - 259s 456ms/step - loss: 0.3245 - accuracy: 0.9078
Epoch 3/5
569/569 [==============================] - 259s 456ms/step - loss: 0.1490 - accuracy: 0.9588
Epoch 4/5
569/569 [==============================] - 259s 456ms/step - loss: 0.0619 - accuracy: 0.9847
Epoch 5/5
64/64 [==============================] - 14s 154ms/step - loss: 0.4185 - accuracy: 0.9081
processing fold # 1
Epoch 1/5
569/569 [==============================] - 277s 455ms/step - loss: 0.3035 - accuracy: 0.9173
Epoch 2/5
569/569 [==============================] - 259s 455ms/step - loss: 0.0528 - accuracy: 0.9873
Epoch 3/5
569/569 [==============================] - 259s 455ms/step - loss: 0.0358 - accuracy: 0.9909
Epoch 4/5
569/569 [==============================] - 259s 455ms/step - loss: 0.0277 - accuracy: 0.9923
Epoch 5/5
64/64 [===================

In [14]:
model.save('bert-k10.h5')

In [13]:
import pickle
with open('bertk10_results.pickle', 'wb') as f:
    pickle.dump(results, f)

In [15]:
    with open('/trainHistoryDict', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)